# NEPOOL IDR Drop

Batches tasks for NEPOOL IDR Drop.

In [1]:
import os
basepath = os.getcwd()
#basepath = 'C:\\Users\wb5888\Documents\la-tools-test\IDR_Drop'
os.chdir(basepath)
print('working in directory {}.'.format(basepath))

from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.keys import Keys
import selenium.webdriver as webdriver
from pandas.io.json import json_normalize
import datetime as dt
import pandas as pd
import numpy as np
import IDRdrop
import EPOwebscrape
import json
import math
import time
import ast
import pprint

print('imported modules.')
pp = pprint.PrettyPrinter(1)

working in directory F:\la-tools-test\IDR_Drop.
webdriver_manager import error can suck my cock.
imported modules.


## Scrape emails

In [ ]:
print('importing emails.')
import emailscrape

os.chdir(os.path.join(basepath, 'Logins'))
print('imported emails.')

importing emails.


## Read .json

In [2]:
os.chdir(os.path.join(basepath, 'Logins'))

with open('email_bodies_12_31_2019.json', 'r') as email:
    email = json.load(email)
    email = json.loads(email)
    
email_df = pd.DataFrame.from_dict(email)
email_df = email_df.T

#sort by date
email_df['date'] = pd.to_datetime(email_df['date'])
email_df.set_index('date', drop = False, inplace = True)
email_df.sort_index(inplace = True, ascending = False)
recent = max(email_df.date) - dt.timedelta(days = 31)
email_df = email_df[[d > recent for d in email_df.date]]
email_df

,accts,date,name,pw,user
date,,,,,
2019-12-31 09:49:27,"[51336834007_200092008, 51939777009_137473005,...",2019-12-31 09:49:27,Walgreen Co.,684xde,wal-engie
2019-12-31 09:46:43,[51281598011_130532006],2019-12-31 09:46:43,40 Richards LLC,be6955,40-engie
2019-12-30 13:32:19,[51486807035_418662008],2019-12-30 13:32:19,Sequel Special Products LLC,79xxet,ssp-engie
2019-12-30 12:48:00,"[51598034007_414582008, 51939937058_744563007,...",2019-12-30 12:48:00,Whitcraft LLC,993gg,craft-engie
2019-12-30 10:44:33,"[8145908003, 8777804027]",2019-12-30 10:44:33,"[HOLY CROSS COLLEGE, HOLY CROSS COLLEGE]",3824,SUEZ_HIST
2019-12-26 13:09:38,"[51426653036_822932008, 51145392007_303891008,...",2019-12-26 13:09:38,National Health Care Associates,4546ee,natc-engie
2019-12-26 13:03:59,[54421791013_036141004],2019-12-26 13:03:59,Amherst College,6bess14,amh-engie
2019-12-23 09:42:25,"[27466290023, 27643570024]",2019-12-23 09:42:25,T-C 33 ARCH ST LLC,zk882,tc-nengie
2019-12-23 09:41:45,"[25658721011, 28616640026, 26706541013]",2019-12-23 09:41:45,ONE POST OFFICE SQUARE,3da46,post-nengie


In [3]:
new_email = email_df

## Get utility

In [4]:
util = []

for a in new_email.accts:
    leading = a[:2]
    if leading == '80':
        util.append('PSNH')
    elif leading == '51':
        util.append('CLP')
    elif leading == '54':
        util.append('WMECO')
    else:
        util.append('NSTAR_NGRID')
        
new_email['util'] = util

In [5]:
output_df = email_df[[u == 'SUEZ_HIST' for u in email_df.user]]
output_df

,accts,date,name,pw,user,util
date,,,,,,
2019-12-30 10:44:33,"[8145908003, 8777804027]",2019-12-30 10:44:33,"[HOLY CROSS COLLEGE, HOLY CROSS COLLEGE]",3824,SUEZ_HIST,NSTAR_NGRID
2019-12-19 15:38:15,[2789541011],2019-12-19 15:38:15,[E INK CORP],3824,SUEZ_HIST,NSTAR_NGRID
2019-12-19 15:34:58,"[2637855079, 3617194003, 9001713005]",2019-12-19 15:34:58,"[ASPEN AEROGELS INC, ASPEN AEROGELS INC, ASPEN...",3824,SUEZ_HIST,NSTAR_NGRID
2019-12-16 13:54:09,"[2552127001, 2796898007]",2019-12-16 13:54:09,"[LOWELL HOUSING AUTHO, LOWELL HOUSING AUTHO]",3824,SUEZ_HIST,NSTAR_NGRID
2019-12-16 13:39:21,[5334624006],2019-12-16 13:39:21,[SEA FREEZE LTD],3824,SUEZ_HIST,NSTAR_NGRID
2019-12-12 14:12:09,[5145630025],2019-12-12 14:12:09,[GREEN GOLDGROUP INC],3824,SUEZ_HIST,NSTAR_NGRID
2019-12-10 12:42:00,[1400895026],2019-12-10 12:42:00,[EQUINIX INC],3824,SUEZ_HIST,NSTAR_NGRID
2019-12-09 14:41:21,"[2692437015, 5257822017, 6285403014, 8779588017]",2019-12-09 14:41:21,"[WILLOWOOD NRSNG HM W, CHRISTIAN HILL CORP, BE...",3824,SUEZ_HIST,NSTAR_NGRID
2019-12-06 16:48:26,"[1385891011, 8863928016]",2019-12-06 16:48:26,"[BRADFORD INDUSTRIES, BRADFORD INDUSTRIES]",3824,SUEZ_HIST,NSTAR_NGRID


# Batch Download IDR

In [7]:
results = []
files = []
beg = time.time()

for row in range(0, len(output_df.user)):
    file = EPOwebscrape.idr_download(output_df.user[row], output_df.pw[row], output_df.accts[row])
    files.append(file)
    results.append(True)
    print('-------------------------------------------')

user:  SUEZ_HIST
password:  3824


KeyboardInterrupt: 

## Filter downloaded files from EPO portal

In [ ]:
results = []
files = []
beg = time.time()

for row in range(0, len(output_df.accts)):
    try:
        file = EPOwebscrape.idr_download(row, output_df)
        files.append(file)
        results.append(True)
        print('done with {} of {}.'.format(row+1, len(output_df.accts)))
        print('---------------------------')
        
    except:
        print('error, accts {}.'.format(output_df.accts[row]))
        files.append(None)
        results.append(False)
        print('done with {} of {}.'.format(row+1, len(output_df.accts)))
        print('---------------------------')

end = time.time()
elapse = round(end - beg, 2)
print('downloaded {} files in {} seconds.'.format(len(output_df.accts), elapse))
output_df['files'] = files
output_df['success'] = results

user:  SUEZ_HIST
password:  3824
set to last  400  days.
logging on...
looking for 2 accts.
login error.
trying search & download...

error, accts ['8145908003', '8777804027'].
done with 1 of 15.
---------------------------
user:  SUEZ_HIST
password:  3824
set to last  400  days.
logging on...
looking for 1 accts.
found items in portal.
trying search & download...

submitting...
error, accts ['2789541011'].
done with 2 of 15.
---------------------------


## Show downloaded files from EPO portal

In [13]:
output_df.to_excel('output_df_results.xlsx')
#output_df = pd.read_excel('output_df_results.xlsx')

In [61]:
output_df

,accts,date,name,user,pw,util,files,success
0,"[51640724076_352382007, 51733934061_347092002,...",2019-10-11 11:36:04,Hartford Healthcare Corporation,harth-engie,78cck,CLP,eversource_2e9c6407-b7f6-4476-9e01-1aa665e843b...,True
1,"[51640724076_352382007, 51733934061_347092002,...",2019-10-11 10:34:43,Hartford Healthcare Corporation,harth-engie,78cck,CLP,eversource_81dc93e5-6fb1-4e33-b189-1e1dd5bc95e...,True


Choose files to split into Raw IDR files.

## Batch process downloaded EPO files into Raw IDRs

In [64]:
readpath = os.path.join(basepath, 'Downloads')
writepath = os.path.join(basepath, 'Raw_IDR')
error_log = []

for i, file in enumerate(output_df.files):
    try:
        if file:
            os.chdir(readpath)
            filedf = pd.read_csv(file, sep = ",", header = 0)
        
            IDRdrop.raw_split(filedf, readpath, writepath, output_df.util[i], output_df.accts[i])
            print('success, file: {}'.format(file))
            
    except:
        print('error, file: {}.'.format(file))

found 4 accounts.
CLP_51640724076_352382007_IDR_RAW.csv  written.
CLP_51733934061_347092002_IDR_RAW.csv  written.
CLP_51801694019_298103007_IDR_RAW.csv  written.
CLP_51970204004_216762005_IDR_RAW.csv  written.
success, file: eversource_2e9c6407-b7f6-4476-9e01-1aa665e843b5_600a1380_hourlycsv.csv
found 4 accounts.
success, file: eversource_81dc93e5-6fb1-4e33-b189-1e1dd5bc95e1_b4b7780f_hourlycsv.csv


## Show Raw IDR files based on utility

Here *rawpath* is directory containing Raw IDRs - 25 most recent will be shown.

In [63]:
rawfiles = IDRdrop.show_dir(writepath, 25)
print(rawfiles)

files found in dir:  /home/jupyter-engiela/la-tools-test/IDR_Drop/Raw_IDR
                                      files                time
0   WMECO_54977381078_126733008_IDR_RAW.csv 2019-10-15 14:11:16
1     CLP_51482323003_190712004_IDR_RAW.csv 2019-10-15 14:11:16
2   WMECO_54202385001_052931000_IDR_RAW.csv 2019-10-15 14:11:16
3   WMECO_54364315051_042703006_IDR_RAW.csv 2019-10-15 14:11:16
4   WMECO_54010781037_952531008_IDR_RAW.csv 2019-10-15 14:11:16
5     CLP_51874553092_860042003_IDR_RAW.csv 2019-10-15 14:11:16
6   WMECO_54480402023_526541004_IDR_RAW.csv 2019-10-15 14:11:16
7     CLP_51549893006_482562007_IDR_RAW.csv 2019-10-15 14:11:16
8     CLP_51436492052_684002003_IDR_RAW.csv 2019-10-15 14:11:16
9     CLP_51801694019_298103007_IDR_RAW.csv 2019-10-15 14:11:15
10    CLP_51970204004_216762005_IDR_RAW.csv 2019-10-15 14:11:15
11    CLP_51640724076_352382007_IDR_RAW.csv 2019-10-15 14:11:15
12    CLP_51733934061_347092002_IDR_RAW.csv 2019-10-15 14:11:15
13                       .ipyn

In [32]:
index = [36]

processfiles = list(rawfiles.files[:13])
print('files to processed: ')
processfiles

files to processed: 


['WMECO_54977381078_126733008_IDR_RAW.csv',
 'CLP_51482323003_190712004_IDR_RAW.csv',
 'WMECO_54202385001_052931000_IDR_RAW.csv',
 'WMECO_54364315051_042703006_IDR_RAW.csv',
 'WMECO_54010781037_952531008_IDR_RAW.csv',
 'CLP_51874553092_860042003_IDR_RAW.csv',
 'WMECO_54480402023_526541004_IDR_RAW.csv',
 'CLP_51549893006_482562007_IDR_RAW.csv',
 'CLP_51436492052_684002003_IDR_RAW.csv',
 'CLP_51801694019_298103007_IDR_RAW.csv',
 'CLP_51970204004_216762005_IDR_RAW.csv',
 'CLP_51640724076_352382007_IDR_RAW.csv',
 'CLP_51733934061_347092002_IDR_RAW.csv']

## Batch filter Raw IDR into IDR files to be dropped

In [33]:
readpath = os.path.join(basepath, 'Raw_IDR')
writepath = os.path.join(basepath, 'IDR')
error_log = []

for dropfile in processfiles:
    try:
        IDRdrop.data_drop(dropfile, readpath, writepath)
        print('success, file: ', dropfile)
    
    except:
        error_log.append(dropfile)
        print("error, file: ", dropfile)

overlap removed at 2/6/2019, for file:
success, file:  WMECO_54977381078_126733008_IDR_RAW.csv
success, file:  CLP_51482323003_190712004_IDR_RAW.csv
success, file:  WMECO_54202385001_052931000_IDR_RAW.csv
success, file:  WMECO_54364315051_042703006_IDR_RAW.csv
success, file:  WMECO_54010781037_952531008_IDR_RAW.csv
overlap removed at 6/27/2019, for file:
success, file:  CLP_51874553092_860042003_IDR_RAW.csv
success, file:  WMECO_54480402023_526541004_IDR_RAW.csv
overlap removed at 2/13/2019, for file:
success, file:  CLP_51549893006_482562007_IDR_RAW.csv
success, file:  CLP_51436492052_684002003_IDR_RAW.csv
success, file:  CLP_51801694019_298103007_IDR_RAW.csv
success, file:  CLP_51970204004_216762005_IDR_RAW.csv
overlap removed at 8/10/2019, for file:
success, file:  CLP_51640724076_352382007_IDR_RAW.csv
success, file:  CLP_51733934061_347092002_IDR_RAW.csv


## Turn Horizontal to Vertical

In [40]:
idr = IDRdrop.show_dir(writepath, 30)

files found in dir:  /home/jupyter-engiela/la-tools-test/IDR_Drop/IDR


In [49]:
vert_files = list(idr.files)
print('working with: ')
vert_files

working with: 


['WMECO_54364315051_042703006_IDR_vert.csv',
 'CLP_51640724076_352382007_IDR_3_vert.csv',
 'CLP_51733934061_347092002_IDR_vert.csv',
 'CLP_51549893006_482562007_IDR_vert.csv',
 'CLP_51436492052_684002003_IDR_vert.csv',
 'WMECO_54202385001_052931000_IDR_vert.csv',
 'CLP_51482323003_190712004_IDR_vert.csv',
 'CLP_51549893006_482562007_IDR_3_vert.csv',
 'WMECO_54977381078_126733008_IDR_vert.csv',
 'CLP_51874553092_860042003_IDR_3_vert.csv',
 'WMECO_54010781037_952531008_IDR_vert.csv',
 'CLP_51801694019_298103007_IDR_vert.csv',
 'CLP_51874553092_860042003_IDR_vert.csv',
 'CLP_51970204004_216762005_IDR.csv',
 'WMECO_54010781037_952531008_IDR.csv',
 'CLP_51874553092_860042003_IDR_3.csv',
 'WMECO_54977381078_126733008_IDR.csv',
 'CLP_51874553092_860042003_IDR.csv',
 'CLP_51549893006_482562007_IDR_3.csv',
 'CLP_51801694019_298103007_IDR.csv',
 'WMECO_54202385001_052931000_IDR.csv',
 'CLP_51436492052_684002003_IDR.csv',
 'CLP_51549893006_482562007_IDR.csv',
 'CLP_51733934061_347092002_IDR.csv',

In [46]:
for v in vert_files:
    IDRdrop.hor_to_vert(v)

FileNotFoundError: [Errno 2] File b'4' does not exist: b'4'